In [1]:
from pathlib import Path
import os

In [2]:
Path().resolve()

PosixPath('/workspace/workspace_pydeequ')

In [3]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pydeequ.checks import Check, CheckLevel
from pydeequ.verification import VerificationSuite, VerificationResult

Deequ is still not supported in spark version: 3.1.2


In [4]:
spark_conf = SparkConf()
spark_conf.setAll(
    [
        ("spark.master", "spark://master:7077"),
        ("spark.app.name", "PyDeequ"),
        ("spark.submit.deployMode", "client"),
        ("spark.ui.showConsoleProgress", "true"),
        ("spark.eventLog.enabled", "false"),
        ("spark.driver.bindAddress", "0.0.0.0"),
        (
            "spark.jars", 
            ",".join([
                "file://{}".format(os.path.join(Path().resolve(), "..", "tmp", "postgresql-42.2.23.jar")),
                "file://{}".format(os.path.join(Path().resolve(), "..", "tmp", "deequ-2.0.0-spark-3.1.jar"))
            ])
        )
    ]
)
spark = (SparkSession
    .builder
    .config(conf=spark_conf)
    .getOrCreate())

In [5]:
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://{}:5432/{}".format(os.getenv("POSTGRES_HOST"), os.getenv("POSTGRES_DB"))) \
    .option("dbtable", "public.demodata") \
    .option("user", os.getenv("POSTGRES_USER")) \
    .option("driver", "org.postgresql.Driver") \
    .load()

In [6]:
check = Check(spark, CheckLevel.Warning, "Review Check")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check.hasSize(lambda x: x > 0) \
        .isUnique("id")  \
        .isComplete("id")  \
        .isContainedIn("country", ["Netherlands", "Spain", "UK", "US"])) \
    .run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()

Python Callback server started!
+------------+-----------+------------+--------------------+-----------------+------------------+
|       check|check_level|check_status|          constraint|constraint_status|constraint_message|
+------------+-----------+------------+--------------------+-----------------+------------------+
|Review Check|    Warning|     Success|SizeConstraint(Si...|          Success|                  |
|Review Check|    Warning|     Success|UniquenessConstra...|          Success|                  |
|Review Check|    Warning|     Success|CompletenessConst...|          Success|                  |
|Review Check|    Warning|     Success|ComplianceConstra...|          Success|                  |
+------------+-----------+------------+--------------------+-----------------+------------------+



In [ ]:
spark.stop()